In [1]:
import requests
import pandas as pd
import json
from urllib.parse import quote

import warnings

# Mute all warnings
warnings.filterwarnings("ignore")

In [14]:
base_url = "https://api.collegefootballdata.com/"

url_add = "teams"

full_url = base_url + url_add

api_token = "gdgvvFI6ZGh3pmGC+3VLf+G7u0ErUjMgC8vDYZxSByy27d2Q7XWbfU3HURW0N6+z"

headers = {
    'Authorization': f'Bearer {api_token}',
    'Content-Type': 'application/JSON'}

In [15]:
response = requests.get(full_url,
                     headers = headers)

data = response.json()

teams = pd.DataFrame(data)

year_list = range(2009,2024,1)


In [56]:
teams['classification'].unique()

array(['fcs', 'ii', 'iii', 'fbs', None], dtype=object)

In [65]:
teams_fbs = teams.loc[teams['classification'] == 'fbs']

In [66]:
teams_fcs = teams.loc[teams['classification'] == 'fcs']

In [67]:
teams_d2 = teams.loc[teams['classification'] == 'ii']

In [68]:
teams_d3 = teams.loc[teams['classification'] == 'iii']

In [49]:
print(len(teams_fbs), len(teams_fcs), len(teams_d2), len(teams_d3))

133 128 169 244


In [69]:
sanctioned_teams = pd.concat([teams_fbs, teams_fcs, teams_d2, teams_d3], ignore_index=True)

In [115]:
roster_information = pd.DataFrame()

In [116]:
counter = 0

import time

start_time = time.time()

for year in year_list:
    for school in teams_fbs['school']:
        mod_team_name = quote(school)
        
        url_add = "roster?team=" + mod_team_name + "&year=" + str(year)
        
        full_url = base_url + url_add
        
        response = requests.get(full_url, headers = headers)
        
        data = response.json()
        
        new_df = pd.DataFrame(data)

        new_df['rosterYear'] = year

        roster_information = pd.concat([roster_information, new_df], ignore_index = True)

        counter += 1

        current_time = time.time()

        time_elapsed = current_time - start_time

        if counter % 100 == 0:
            print(counter, year, len(roster_information), time_elapsed)


100 2009 4278 66.15104341506958
200 2010 8580 135.87768125534058
300 2011 13130 206.31269931793213
400 2012 17672 277.04093313217163
500 2012 22504 348.53480291366577
600 2013 27634 419.2055072784424
700 2014 33319 488.8234007358551
800 2015 39494 557.3821003437042
900 2015 46375 631.0240659713745
1000 2016 54222 704.8144040107727
1100 2017 62708 775.6538963317871
1200 2018 71438 845.5027928352356
1300 2018 80710 916.3891086578369
1400 2019 91411 987.5397417545319
1500 2020 103120 1059.8272323608398
1600 2021 115188 1135.1508574485779
1700 2021 126583 1206.788650751114
1800 2022 138410 1276.1581861972809
1900 2023 150232 1346.6487770080566


In [117]:
len(roster_information['id'].unique())

54289

In [8]:
#pip install cfbda

teams.head(25)

In [114]:
roster_information.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162765 entries, 0 to 162764
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                162765 non-null  object 
 1   first_name        162758 non-null  object 
 2   last_name         162757 non-null  object 
 3   team              162765 non-null  object 
 4   weight            118758 non-null  float64
 5   height            160512 non-null  float64
 6   jersey            159094 non-null  float64
 7   year              108828 non-null  float64
 8   position          161102 non-null  object 
 9   home_city         160323 non-null  object 
 10  home_state        159698 non-null  object 
 11  home_country      160312 non-null  object 
 12  home_latitude     157457 non-null  object 
 13  home_longitude    157457 non-null  object 
 14  home_county_fips  157249 non-null  object 
 15  recruit_ids       162765 non-null  object 
 16  rosterYear        16

In [119]:
roster_information2 = roster_information #leaving a backup

In [121]:
roster_information = roster_information2

In [122]:
roster_information = roster_information.rename(columns = {
                                                'id':'playerId',
                                                'first_name':'firstName',
                                                'last_name':'lastName',
                                                'team':'collegeTeam',
                                                'year':'schoolYear',
                                                'home_city':'homeCity',
                                                'home_state':'homeState',
                                                'home_country':'homeCountry',
                                                'home_latitude':'homeLatitude',
                                                'home_longitude':'homeLongitude',
                                                'home_county_fips':'homeCountyFips',
                                                'recruit_ids':'recruitIds'

})

In [132]:
roster_information = roster_information.drop(columns= ['recruitIds'])

In [124]:
from google.cloud import bigquery
import os

In [125]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/gshoda/key.json'

In [126]:
client = bigquery.Client()

In [127]:
dataset = 'sp24-i535-gshoda-nfldraft.NFL_Draft'
table_ref = client.dataset(dataset).table('College_Players')

In [128]:
import pandas_gbq

In [133]:
project_id = "sp24-i535-gshoda-nfldraft"
table_name = 'NFL_Draft.College_Players'

pandas_gbq.to_gbq(roster_information, table_name, project_id = project_id, if_exists='replace')

In [134]:
teams_fbs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, 3 to 879
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              133 non-null    int64 
 1   school          133 non-null    object
 2   mascot          133 non-null    object
 3   abbreviation    133 non-null    object
 4   alt_name1       18 non-null     object
 5   alt_name2       133 non-null    object
 6   alt_name3       133 non-null    object
 7   conference      133 non-null    object
 8   classification  133 non-null    object
 9   color           133 non-null    object
 10  alt_color       131 non-null    object
 11  logos           133 non-null    object
 12  twitter         132 non-null    object
 13  location        133 non-null    object
dtypes: int64(1), object(13)
memory usage: 15.6+ KB


In [136]:
teams_fbs = teams_fbs.drop(columns = ['logos'])

In [137]:
teams_fbs = teams_fbs.rename(columns = {
                                    'alt_name1':'alternateName1',
                                    'alt_name2':'alternateName2',
                                    'alt_name3':'alternateName3',
                                    'alt_color':'alternateColor'
})

In [139]:
expanded_location = pd.json_normalize(teams_fbs['location'])

In [142]:
teams_fbs_expanded = pd.concat([teams_fbs.drop(columns=['location']), expanded_location], axis = 1)

In [144]:
teams_fbs_expanded.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 247 entries, 3 to 132
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                133 non-null    float64
 1   school            133 non-null    object 
 2   mascot            133 non-null    object 
 3   abbreviation      133 non-null    object 
 4   alternateName1    18 non-null     object 
 5   alternateName2    133 non-null    object 
 6   alternateName3    133 non-null    object 
 7   conference        133 non-null    object 
 8   classification    133 non-null    object 
 9   color             133 non-null    object 
 10  alternateColor    131 non-null    object 
 11  twitter           132 non-null    object 
 12  venue_id          133 non-null    float64
 13  name              133 non-null    object 
 14  city              133 non-null    object 
 15  state             133 non-null    object 
 16  zip               133 non-null    object 
 17  co

In [145]:
teams_fbs_expanded = teams_fbs_expanded.rename(columns = {
                                                            'venue_id':'venueId',
                                                            'country_code':'countryCode',
                                                            'year_constructed':'yearConstructed'
})

In [151]:
teams_fbs_expanded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 247 entries, 3 to 132
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               133 non-null    float64
 1   school           133 non-null    object 
 2   mascot           133 non-null    object 
 3   abbreviation     133 non-null    object 
 4   alternateName1   18 non-null     object 
 5   alternateName2   133 non-null    object 
 6   alternateName3   133 non-null    object 
 7   conference       133 non-null    object 
 8   classification   133 non-null    object 
 9   color            133 non-null    object 
 10  alternateColor   131 non-null    object 
 11  twitter          132 non-null    object 
 12  venueId          133 non-null    float64
 13  name             133 non-null    object 
 14  city             133 non-null    object 
 15  state            133 non-null    object 
 16  zip              133 non-null    object 
 17  countryCode      132 

In [155]:
teams_fbs_expanded['yearConstructed'] = pd.to_numeric(teams_fbs_expanded['yearConstructed'], errors = 'coerce').fillna('0').astype(int)
teams_fbs_expanded['id'] = pd.to_numeric(teams_fbs_expanded['id'], errors = 'coerce').fillna('0').astype(int)
teams_fbs_expanded['venueId'] = pd.to_numeric(teams_fbs_expanded['venueId'], errors = 'coerce').fillna('0').astype(int)
teams_fbs_expanded['capacity'] = pd.to_numeric(teams_fbs_expanded['capacity'], errors = 'coerce').fillna('0').astype(int)

In [156]:
teams_fbs_expanded = teams_fbs_expanded.rename(columns = {
                    'id':'teamId'
})

In [159]:
teams_fbs_expanded['grass'] = teams_fbs_expanded['grass'].fillna('0').astype(int).astype(bytes)
teams_fbs_expanded['dome'] = teams_fbs_expanded['dome'].fillna('0').astype(int).astype(bytes)

In [152]:
#table_ref = client.dataset(dataset).table('College_Teams')

In [160]:
project_id = "sp24-i535-gshoda-nfldraft"
table_name = 'NFL_Draft.College_Teams'

pandas_gbq.to_gbq(teams_fbs_expanded, table_name, project_id = project_id, if_exists='replace')